# 共起ネットワーク分析スクリプト

このスクリプトは、事前に解析されたテキストデータ（.spacyファイル）を読み込み、共起行列を作成して共起ネットワークを構築する。得られたネットワークは、Pyvisライブラリを用いてインタラクティブに可視化される。また、ネットワークのノードとエッジの情報がCSVファイルとして出力される。

## 主な機能

1. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）の読み込み
2. 共起行列の作成
  - ユーザー指定の出現回数の範囲内でのフィルタリング
  - トークンの最小長の指定
3. エッジの重み計算
  - 共起頻度、Jaccard係数、Dice係数、Simpson係数から選択
4. 共起ネットワークの構築
  - エッジの重みの範囲指定
  - 出力するエッジ数の指定
  - 独立したエッジの除去
  - コミュニティ検出
  - ノードの重み計算（中心性または出現頻度）
5. ネットワークの可視化（Pyvisライブラリ）
  - ノードとエッジのサイズ調整
  - ノードの属性情報の表示
  - インタラクティブな操作
6. ネットワークのノードとエッジ情報のCSV出力

## 使用方法

1. 必要なライブラリをインストールする（`spacy`, `networkx`, `pyvis`など）
2. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）を用意する
3. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する
  - `main_directory`: 親ディレクトリへの相対パス
  - `subdirectories`: 読み込む子ディレクトリ名のリスト
  - `nlp_model`: 解析に利用したモデル名
  - `selected_file_names`: 特定のファイルのみを読み込む場合はここにtxtファイル名を追加（空リストの場合は全ファイルを読み込む）
4. 共起行列の作成に関するパラメーターを設定する
  - `min_freq`, `max_freq`: 出現回数の下限と上限
  - `token_length`: トークンの最小長
5. エッジの重み計算方法を選択する
  - `weight_mode`: 'cooccurrence', 'jaccard', 'dice', 'simpson'から選択
6. 共起ネットワークの構築に関するパラメーターを設定する
  - `weight_lower_cutoff`, `weight_upper_cutoff`: エッジの重みの範囲
  - `num_edges_to_output`: 出力するエッジの数（Noneの場合は全て出力）
  - `max_degree`: カットオフする独立したエッジの数（Noneの場合は全て出力）
  - `node_multiplier`, `edge_multiplier`: ノードとエッジのサイズ調整係数
   - `node_weight_mode`: ノードの重みを計算する方法（`'centrality'` または `'frequency'`）
     - `'centrality'`: ネットワーク内でのノードの重要度（中心性）に基づく
     - `'frequency'`: テキスト内での単語の出現頻度に基づく
7. スクリプトを実行する
8. 出力されたHTMLファイル（`network.html`）をウェブブラウザで開き、インタラクティブなネットワーク可視化を explore する
9. 出力されたCSVファイル（`network_edges.csv`, `network_nodes.csv`）を必要に応じて他のツールで分析する

## 注意事項

- 大規模なデータセットを扱う場合、処理に時間がかかる可能性がある
- パラメーターの設定によっては、ネットワークが複雑になりすぎて解釈が困難になる場合がある
- このスクリプトは、事前に解析されたテキストデータ（.spacyファイル）を必要とする。生のテキストデータを扱う場合は、別途前処理が必要である

# **ライブラリのインポート(最初に実行)**

In [1]:
import os
import csv
import spacy
from spacy.tokens import DocBin
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import networkx as nx
from networkx.algorithms import community
from pyvis.network import Network

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["sub1"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データ読み込みの実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **テキストデータの前処理（品詞指定, ストップワード除去）**
## ユーザーが設定するパラメーター

In [33]:
# 抽出する品詞リストを設定
include_pos = ["NOUN", "PROPN", "ADJ", "ADV"]

# ストップワードの設定
custom_stopwords_path = 'stopwords.txt'  # ユーザー定義のストップワードファイルのパス
use_spacy_stopwords = True  # spacyのデフォルトストップワードを使用するかどうか

## 前処理の実行

In [ ]:
def load_stopwords(file_path):
    """
    ストップワードをファイルから読み込む関数

    Args:
        file_path (str): ストップワードが記載されたファイルのパス

    Returns:
        set: ストップワードの集合
    """
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().splitlines())
    else:
        stopwords = set()
        print("警告: ストップワードファイルが見つかりません。ストップワードは使用されません。")
    return stopwords

def extract_words(sent, pos_tags, stopwords):
    """
    文から指定した品詞のトークンを抽出し、ストップワードを除去する関数

    Args:
        sent (spacy.tokens.span.Span): spaCyの文オブジェクト
        pos_tags (list): 抽出対象の品詞リスト
        stopwords (set): ストップワードの集合

    Returns:
        list: 抽出されたトークンのリスト
    """
    words = [token.lemma_ for token in sent if token.pos_ in pos_tags and token.lemma_ not in stopwords]
    return words

all_sents = []  # 前処理後の文を格納するリスト
spacy_stopwords = nlp.Defaults.stop_words  # spaCyのモデルのストップワードをロード
custom_stopwords = load_stopwords(custom_stopwords_path)  # ユーザー定義のストップワードを読み込む

if use_spacy_stopwords:
    combined_stopwords = spacy_stopwords | custom_stopwords  # spacyのデフォルトストップワードとユーザー定義のストップワードの和集合を取る
    print("spacyのデフォルトストップワードを使用します。")
else:
    combined_stopwords = custom_stopwords  # ユーザー定義のストップワードのみを使用
    print("spacyのデフォルトストップワードは使用しません。")

for doc in filtered_docs.values():  # 読み込んだ文書に対して処理を行う
    for sent in doc.sents:  # 各文に対して処理を行う
        words = extract_words(sent, include_pos, combined_stopwords)
        all_sents.append(' '.join(words)) # 前処理後のトークンを空白区切りで連結し、リストに格納

print(all_sents[:10])  # 前処理後の文を10件表示

# **共起行列の作成**
## ユーザーが設定するパラメーター

In [ ]:
# エッジの重みを計算する方法を選択(cooccurrence, jaccard, dice, simpson)
weight_mode = 'cooccurrence' 

# 上限と下限の出現回数を設定(以上/以下)
min_freq = 2
max_freq = 100000

# 文の最小出現回数を設定(jaccard, dice, simpsonの場合のみ有効)
min_sentence_count = 1

# トークンの最小長を設定(N文字以上を対象にする、1文字の単語を除外する場合は2以上に設定)
token_length = 1

## 共起行列作成の実行

In [ ]:
def count_cooccurrence(sents, min_freq, max_freq, token_length):
    """
    共起行列を生成する関数

    Args:
        sents (list): 文のリスト
        min_freq (int): 出現回数の下限
        max_freq (int): 出現回数の上限
        token_length (int): トークンの最小長

    Returns:
        tuple: 以下の要素を持つタプル
            - words (list): 語彙のリスト
            - token_freq_dict (dict): 語彙の出現回数の辞書
            - Xc (scipy.sparse.csr_matrix): 共起行列
    """
    token_pattern = f'\\b\\w{{{token_length},}}\\b'  # token_length文字以上の単語を抽出する正規表現パターン
    vectorizer = CountVectorizer(token_pattern=token_pattern) # CountVectorizerの初期化
    X = vectorizer.fit_transform(sents) # 前処理後の文を行列に変換
    
    # 語彙と出現回数を取得
    feature_names = vectorizer.get_feature_names_out()
    frequencies = X.sum(axis=0).A1
    
    # 出現回数に基づいて語彙をフィルタリング
    mask = (min_freq <= frequencies) & (frequencies <= max_freq) # 出現回数がmin_freq以上max_freq以下の語彙を選択
    filtered_vocab = [feat for feat, m in zip(feature_names, mask) if m] # フィルタリングされた語彙を取得
    
    # フィルタリングされた語彙を使って再度CountVectorizerを適用
    filtered_vectorizer = CountVectorizer(vocabulary=filtered_vocab, token_pattern=token_pattern) # フィルタリングされた語彙を使ってCountVectorizerを初期化
    filtered_X = filtered_vectorizer.fit_transform(sents) # フィルタリングされた語彙を使って文を行列に変換
    
    # 共起行列の生成
    words = filtered_vectorizer.get_feature_names_out()
    token_freq = filtered_X.sum(axis=0).A1
    token_freq_dict = dict(zip(words, token_freq))
    filtered_X[filtered_X > 0] = 1  # 各要素が1以上の場合は1に変換
    Xc = (filtered_X.T * filtered_X)  # 共起行列の生成
    
    return words, token_freq_dict, Xc

def cooccurrence_weights(words, Xc):
    """
    共起行列から単語間の重みを計算する関数

    Args:
        words (list): 語彙のリスト
        Xc (scipy.sparse.csr_matrix): 共起行列

    Returns:
        list: (単語1, 単語2, 重み) のタプルのリスト
    """
    Xc.setdiag(0) # 対角成分を0にする
    Xc_max = Xc.max() # 共起回数の最大値を取得
    weights = [(words[i], words[j], Xc[i,j] / Xc_max) for i, j in zip(*Xc.nonzero()) if i < j] # 上三角行列のみ取得し、正規化を行う
    return weights

def cooccurrence_similarity(words, sents, Xc, mode, min_sentence_count=1):
    """
    共起行列から単語間の類似度を計算する関数

    Args:
        words (list): 語彙のリスト
        sents (list): 文のリスト
        Xc (scipy.sparse.csr_matrix): 共起行列
        mode (str): 類似度の計算方法 ('jaccard', 'dice', 'simpson')
        min_sentence_count (int): 単語が出現する最小文数の閾値 . Defaults to 1.

    Returns:
        tuple: 以下の要素を持つタプル
            - weights (list): (単語1, 単語2, 類似度) のタプルのリスト
            - csv_data (list): CSV出力用のデータのリスト
    """
    word_to_sents = {word: set() for word in words}
    for i, sent in enumerate(sents):
        for word in sent.split():
            if word in word_to_sents:
                word_to_sents[word].add(i)

    weights = []
    csv_data = []
    for i, j in zip(*Xc.nonzero()):
        if i < j: 
            set_i = word_to_sents[words[i]]
            set_j = word_to_sents[words[j]]
            count1 = len(set_i)
            count2 = len(set_j)

            if count1 > min_sentence_count and count2 > min_sentence_count:
                intersection = len(set_i & set_j)

                if mode == 'jaccard':
                    union = len(set_i | set_j)
                    if union > 0:
                        jaccard = intersection / union
                        csv_data.append([words[i], words[j], intersection, count1, count2, union, jaccard])
                        weights.append((words[i], words[j], jaccard))
                elif mode == 'dice':
                    dice = (2 * intersection) / (count1 + count2)
                    csv_data.append([words[i], words[j], intersection, count1, count2, dice])
                    weights.append((words[i], words[j], dice))
                elif mode == 'simpson':
                    simpson = intersection / min(count1, count2)
                    csv_data.append([words[i], words[j], intersection, count1, count2, simpson])
                    weights.append((words[i], words[j], simpson))

    return weights, csv_data

def calculate_weights(words, all_sents, Xc, weight_mode, min_sentence_count=1):
    """
    エッジの重み計算の方法に応じて関数を選択し、重みを計算する関数

    Args:
        words (list): 語彙のリスト
        all_sents (list): 全ての文のリスト
        Xc (scipy.sparse.csr_matrix): 共起行列
        weight_mode (str): 重み計算の方法 ('cooccurrence', 'jaccard', 'dice', 'simpson')
        min_sentence_count (int): 単語が出現する最小文数の閾値 . Defaults to 1.

    Returns:
        list: (単語1, 単語2, 重み) のタプルのリスト
    
    Raises:
        ValueError: 不明な重み計算方法が指定された場合
    """
    if weight_mode == 'cooccurrence':
        weights = cooccurrence_weights(words, Xc)
        csv_header = ['word1', 'word2', 'cooccurrence']
    elif weight_mode in ['jaccard', 'dice', 'simpson']:
        weights, csv_data = cooccurrence_similarity(words, all_sents, Xc, weight_mode, min_sentence_count)
        if weight_mode == 'jaccard':
            csv_header = ['word1', 'word2', 'intersection', 'count1', 'count2', 'union', 'jaccard']
        elif weight_mode == 'dice':
            csv_header = ['word1', 'word2', 'intersection', 'count1', 'count2', 'dice']
        else:  # simpson
            csv_header = ['word1', 'word2', 'intersection', 'count1', 'count2', 'simpson']
    else:
        raise ValueError(f"不明な重み計算方法: {weight_mode}")
    
    weights.sort(key=lambda x: x[2], reverse=True)  # 重みでソート
    weights = weights[:10000]  # 上位10000件のみを取得
    
    if weight_mode != 'cooccurrence':
        csv_data.sort(key=lambda x: x[-1], reverse=True)  # 重みでソート
        csv_data = csv_data[:10000]  # 上位10000件のみを取得
        
        with open(f"{weight_mode}_edge.csv", 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(csv_header)
            writer.writerows(csv_data)
    else:
        with open('cooccurrence_edge.csv', 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(csv_header)
            writer.writerows(weights)
    
    return weights

# 共起行列の作成
words, token_freq_dict, Xc = count_cooccurrence(all_sents, min_freq, max_freq, token_length)
# 共起行列のトークン情報(ストップワードや出現回数のフィルタリング後の語彙)
with open("filtered_tokens.txt", "w", encoding="utf-8") as f:
    for token in words:
        f.write(token + "\n")

# エッジの重みを計算
weights = calculate_weights(words, all_sents, Xc, weight_mode, min_sentence_count)

# 情報の表示
print(f"共起行列のサイズ: {Xc.shape}") 
print(f"選択された重み計算方法: {weight_mode}")
print(f"エッジ数: {len(weights)}")
print(weights[:10])  # 上位10件のエッジを表示

# **共起ネットワークの作成**
## ユーザーが設定するパラメーター

In [55]:
# エッジの重みの範囲を指定(以上/以下)
weight_lower_cutoff = 0.02
weight_upper_cutoff = 1

# 出力するエッジの数を指定（None の場合は全て出力）
num_edges_to_output = 1000

# カットオフする独立したエッジの数を指定(0の場合は全て出力)
max_degree = 1

# ノードとエッジのサイズを調整するための係数
node_multiplier = 50
edge_multiplier = 20

# ノードの重みを計算する方法を選択('centrality' または 'frequency')
node_weight_mode = 'centrality'  

## 実行セル

In [ ]:
def filter_edges(weights, lower_cutoff, upper_cutoff, num_edges=None):
    """
    エッジの重みの範囲に基づいてエッジをフィルタリングする関数

    Args:
        weights (list): (単語1, 単語2, 重み) のタプルのリスト
        lower_cutoff (float): 重みの下限値
        upper_cutoff (float): 重みの上限値
        num_edges (int, optional): 出力するエッジの数. Noneの場合は全て出力. Defaults to None.

    Returns:
        list: フィルタリングされたエッジのリスト
    """
    filtered_weights = [edge for edge in weights if lower_cutoff <= edge[2] <= upper_cutoff] # 指定された範囲のエッジを取得(下限以上,上限以下)
    sorted_weights = sorted(filtered_weights, key=lambda x: x[2], reverse=True) # 重みでソート
    
    # エッジ数の指定がある場合は上位のエッジのみを返す
    if num_edges is None or num_edges >= len(sorted_weights): 
        return sorted_weights
    else:
        return sorted_weights[:num_edges]

def remove_isolated_edges(G, max_degree):
    """
    グラフから独立したエッジを削除する関数

    Args:
        G (networkx.Graph): NetworkXのグラフオブジェクト
        max_degree (int): カットオフする独立したエッジの最大次数

    Returns:
        networkx.Graph: 独立したエッジを削除したグラフ
    """
    edges_to_remove = [] # 削除するエッジのリスト
    for u, v in G.edges(): # 全てのエッジに対して処理を行う
        if G.degree(u) <= max_degree and G.degree(v) <= max_degree: # 両端のノードの次数がmax_degree以下の場合
            edges_to_remove.append((u, v)) # エッジを削除するリストに追加
    G.remove_edges_from(edges_to_remove) # エッジの削除
    
    # 孤立したノードの削除
    isolated_nodes = [node for node in G.nodes() if G.degree(node) == 0] # 次数が0のノードを取得
    G.remove_nodes_from(isolated_nodes) # 孤立したノードを削除
    
    return G

def create_network_with_communities(words, weights, max_degree, node_weight_mode):
    """
    共起ネットワークを作成し、コミュニティを検出する関数

    Args:
        words (list): 語彙のリスト
        weights (list): (単語1, 単語2, 重み) のタプルのリスト
        max_degree (int): カットオフする独立したエッジの最大次数
        node_weight_mode (str): ノードの重み計算方法 ('centrality' または 'frequency')

    Returns:
        networkx.Graph: コミュニティ情報を含む共起ネットワーク
    """
    G = nx.Graph()
    filtered_weights = filter_edges(weights, weight_lower_cutoff, weight_upper_cutoff, num_edges_to_output)
    G.add_weighted_edges_from(filtered_weights)

    # エラー処理
    if not G.nodes() or not G.edges():
        raise ValueError("ネットワークが空です。エッジの重みの範囲を調整してください。")

    # 独立したエッジの削除
    G = remove_isolated_edges(G, max_degree)

    # コミュニティ検出
    communities = community.greedy_modularity_communities(G)
    community_dict = {node: cid for cid, com in enumerate(communities) for node in com}

    # **ノードの重み計算**
    if node_weight_mode == 'centrality':
        # 中心性に基づく重み計算
        centrality = nx.degree_centrality(G)
        max_centrality = max(centrality.values(), default=1)
        normalized_weights = {node: centrality[node] / max_centrality for node in G.nodes()}
    elif node_weight_mode == 'frequency':
        # 出現頻度に基づく重み計算
        node_freq = {node: token_freq_dict.get(node, 0) for node in G.nodes()}
        max_freq = max(node_freq.values(), default=1)
        normalized_weights = {node: node_freq[node] / max_freq for node in G.nodes()}
    else:
        raise ValueError(f"不明なノード重み計算方法: {node_weight_mode}")

    # ノード属性の設定
    for node in G.nodes():
        G.nodes[node]['weight'] = normalized_weights[node]
        G.nodes[node]['community'] = community_dict.get(node, 0)

    return G

def nx2pyvis_G(G, node_multiplier, edge_multiplier, min_size=0.01):
    """
    NetworkXのグラフをPyvisのグラフに変換する関数

    Args:
        G (networkx.Graph): NetworkXのグラフオブジェクト
        node_multiplier (float): ノードサイズの調整係数
        edge_multiplier (float): エッジサイズの調整係数
        min_size (float, optional): ノードの最小サイズ. Defaults to 0.01.

    Returns:
        pyvis.network.Network: Pyvisのグラフオブジェクト
    """
    # Pyvisのグラフオブジェクトの初期化
    pyvis_G = Network(height="850px", width="100%", bgcolor="#222222", font_color="white", filter_menu=True) 
    
    # ノードに情報を追加(NetworkXのノード属性をPyvisのノード属性に変換)
    for node, attrs in G.nodes(data=True): # 全てのノードに対して処理を行う(ノード名, 属性の辞書)
        size = max(node_multiplier * attrs.get('weight', 1), min_size) # ノードのサイズを計算
        pyvis_G.add_node(node, title=node, size=size, group=attrs.get('community', 0)) # ノードを追加
    
    # エッジに情報を追加(NetworkXのエッジ属性をPyvisのエッジ属性に変換)
    for node1, node2, attrs in G.edges(data=True):
        weight = attrs.get('weight', 1) 
        title = f"{node1} - {node2}\nWeight: {weight:.2f}" 
        pyvis_G.add_edge(node1, node2, title=title, width=edge_multiplier * weight)
    
    pyvis_G.force_atlas_2based()
    
    # 隣接ノードの情報を取得
    neighbor_map = pyvis_G.get_adj_list()
    
    # 各ノードの情報を更新
    for node in pyvis_G.nodes:
        weight_info = G.nodes[node['id']].get('weight', 1)
        title_text = f"{node['id']}\nWeight: {weight_info:.2f}\nCommunity: {node['group']}\nNeighbors:\n"
        title_text += "\n".join(neighbor_map[node['id']])
        node['title'] = title_text
    
    pyvis_G.show_buttons(filter_=['physics', 'nodes']) # 設定ボタンの表示
    
    return pyvis_G

# ネットワークの作成
G = create_network_with_communities(words, weights, max_degree, node_weight_mode)

# Pyvisでの可視化
pyvis_G = nx2pyvis_G(G, node_multiplier, edge_multiplier)
pyvis_G.save_graph("network.html")

# ネットワークのノードとエッジ数を表示
print(f"ノード数: {G.number_of_nodes()}")
print(f"エッジ数: {G.number_of_edges()}")

# **ネットワーク情報の出力(CSVファイル)**

In [ ]:
# エッジ情報のCSV出力
with open("network_edges.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Source", "Target", "Weight"])
    for u, v, attrs in G.edges(data=True):
        writer.writerow([u, v, attrs.get("weight", 1)])

# ノード情報のCSV出力
with open("network_nodes.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Community", "Node", "Frequency", "Degree", "Weight"])
    for node, attrs in G.nodes(data=True):
        writer.writerow([attrs.get("community", 0), node, token_freq_dict.get(node, 0), G.degree(node), attrs.get("weight", 1)])

print("ネットワークのデータをCSVファイルに保存しました。")